In [0]:
%run ./config_and_imports

### Forecast macros - 5y horizon

In [0]:
logging.getLogger('py4j').setLevel(logging.ERROR)

def train_and_score(history_pd, periods):
  model = Prophet(
    interval_width=0.95,
    growth='linear'
    )

  model.fit(history_pd)

  future_pd = model.make_future_dataframe( 
    periods=periods, 
    freq='MS',
    include_history=True
    )
  
  forecast_pd = model.predict(future_pd)
  
  return(forecast_pd)

In [0]:
variables = ["unemploymentrate", "HousePriceIndex", "CPIInflationRate"]
predictions = pd.DataFrame()

for var in variables:
  query = f"select date as ds, {var} as y from raw_macro"
  pred_df = spark.sql(query)
  preds = train_and_score(pred_df.toPandas(), 60)
  preds = preds[["ds", "yhat"]]
  preds.rename(columns={"yhat": var+"_pred"}, inplace=True)
  print(preds)
  if predictions.empty:
    predictions = preds
  else:
    predictions = predictions.merge(preds, on="ds")

In [0]:
display(predictions)

In [0]:
preds_pdf = spark.createDataFrame(predictions)
preds_pdf.write.format("delta").mode("overwrite").saveAsTable("gold_macro_predictions")